In [1]:
# !pip3 install --user tensorflow=='2.0.0a0'
# !pip3 install --user sklearn

In [2]:
import os
import joblib

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Activation, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score, classification_report

np.random.seed(seed=17)

### Simulation Subset Selection
- Use the results summary file to select a subset of simulations for model training and testing
    - Randomly select n non-fail cases and n fail cases
    - Use the simulation ids to read in the corresponding simulation data files (pickled numpy arrays)


In [3]:
# Import simulation results
results = pd.read_csv('processed_results_summary.csv', keep_default_na=False)
results.head()

,Unnamed: 0,Simulation no#,Line,Load,Wind1,Wind2,Wind3,flag,Sequence,n_failures,t_fail,t_first_fail,dt_fail,failed component_type,failed_component_type,cascade,fail
0,0,1,Line 04 - 05,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
1,18,10,Line 26 - 28,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
2,198,100,Line 09 - 39,0.7,0.0,0.0,0.4,0,[],0,,,,,,False,False
3,1998,1000,Line 22 - 23,0.7,0.0,0.8,1.0,0,[],0,,,,,,False,False
4,19998,10000,Line 10 - 11,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False


### Data Generator

- Create a data generator to read in training, validation and test data by batches
- Model input formats:  
    x -> (n_batch, n_time_steps, n_features)  
    y -> (n_batch, 1)

In [4]:
DATA_PATH = '/output/02_preprocessed_numpy'
files = os.listdir(DATA_PATH)

In [5]:
import random

# Get the simulation ids of sims with and without failures
fail_ids_full = results[results['flag']==1]['Simulation no#'].tolist()
non_fail_ids_full = results[results['flag']==0]['Simulation no#'].tolist()

fail_ids = results[results['flag']==1]['Simulation no#'].tolist()
non_fail_ids = results[results['flag']==0]['Simulation no#'].tolist()

# Randomly select fail and non-fail ids
fail_ids = np.random.choice(fail_ids, size=len(fail_ids), replace=False)
non_fail_ids = np.random.choice(non_fail_ids, size=20000, replace=False)

# Get full filepaths
paths_fail, paths_non_fail = [], []

num_fail_cases = 4000 # this is the maximum value
num_non_fail_cases = 8000 # this is the maximum value

for file in files:
    file_id = int(file.split('.pkl')[0])

    if file_id in fail_ids and len(paths_fail) < num_fail_cases:
        paths_fail.append(os.path.join(DATA_PATH, file))

    if file_id in non_fail_ids and len(paths_non_fail) < num_non_fail_cases:
        paths_non_fail.append(os.path.join(DATA_PATH, file))

print(len(paths_fail), len(paths_non_fail))

num_fail_cases_actual = len(paths_fail) 
num_non_fail_cases_actual = len(paths_non_fail)
total_cases = num_fail_cases_actual+num_non_fail_cases_actual
# indices = np.random.permutation(total_cases) # random permutation of np.arange(total_cases)
# filepaths = [None]*(total_cases)
# filepaths[indices[:num_fail_cases_actual]] = paths_fail
# filepaths[indices[num_fail_cases_actual:]] = paths_non_fail

filepaths = paths_fail + paths_non_fail # concatenating two lists

random.shuffle(filepaths) # shufflinf list in place

train_fraction = 0.6
val_fraction = 0.1

train_length = int(train_fraction*total_cases)
val_length = int(val_fraction*total_cases)
test_length = total_cases - train_length - val_length

train_paths = filepaths[:train_length]
val_paths = filepaths[train_length:train_length+val_length]
test_paths = filepaths[train_length+val_length:]

4000 8000


In [6]:
len(paths_fail + paths_non_fail)

12000

In [7]:
def data_generator(batch_size, filepaths, length, shift):
    count = 0
    X, y = [], []

    for file in filepaths:
        # Load simulation file as numpy array
        arr = joblib.load(os.path.join(DATA_PATH, file))
        simu_number = int(str(file).split("/")[-1].split(".")[0]) # getting simulation number
#         print(int(simu_number in fail_ids_full))
#         print(int(simu_number))

        # Windowing - create evenly sized samples
        end_index = arr.shape[0] - length

        for i in range(0, end_index, shift):
#             y.append(int(arr[i:(i+length), 259][0]))
            y.append(int(simu_number in fail_ids_full))  # if simulation number is in failed list, returns 1 else 0
            X.append(arr[i:(i+length), 1:].flatten()) # CONVERTING TO A SINGLE COLUMN
            count += 1

            if count >= batch_size:
                yield (np.array(X).astype(np.float32), np.array(y))
                X, y = [], []
                count = 0

In [8]:
# testing data_generator() 
increm = 0
for k in data_generator(16, val_paths[4:5], 100, 25):
    increm = increm +1
    print((k[1]))
    if increm > 10:
        break

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Model Training and Evaluation
- Train a single layer LSTM model
- Evaluate on test data: accuracy, precision, recall, F1 score

In [9]:
# logistic
EPOCH_SIZE = 10

# Number of time steps and number of features
n_time_steps = 1
n_feats = 259

model = Sequential()
model.add(Input(shape=(n_time_steps*n_feats,)))
# model.add(LSTM(50, input_shape = (n_time_steps, n_feats)))
model.add(Dense(1, activation = 'sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 260       
Total params: 260
Trainable params: 260
Non-trainable params: 0
_________________________________________________________________


In [10]:

# Start training
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint(
    filepath='checkpoint',
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True
)

model.fit_generator(
    data_generator(256, train_paths, n_time_steps, 25),
    validation_data=data_generator(256, val_paths, n_time_steps, 25),
    validation_steps=6,
    epochs=EPOCH_SIZE,
    steps_per_epoch=100,
#     callbacks=[early_stop, checkpoint]
)

model.save('model_logistic')

Epoch 1/10


  1/100 [..............................] - ETA: 2:48 - loss: 2.1763 - accuracy: 0.2695

  2/100 [..............................] - ETA: 1:44 - loss: 2.4584 - accuracy: 0.1348

  3/100 [..............................] - ETA: 1:40 - loss: 2.0294 - accuracy: 0.2552

  4/100 [>.............................] - ETA: 1:39 - loss: 1.8743 - accuracy: 0.2871

...

 60/100 [=================>............] - ETA: 28s - loss: 0.3180 - accuracy: 0.8691

 61/100 [=================>............] - ETA: 28s - loss: 0.3158 - accuracy: 0.8701

 62/100 [=================>............] - ETA: 27s - loss: 0.3184 - accuracy: 0.8693

 63/100 [=================>............] - ETA: 26s - loss: 0.3194 - accuracy: 0.8689

 64/100 [==================>...........] - ETA: 26s - loss: 0.3228 - accuracy: 0.8683

In [0]:
test_y, pred_y = [], []

for X, y in data_generator(256, test_paths, n_time_steps, 25):
    test_y.append(y)
    pred_y.append(model.predict(X))

test_y = np.concatenate(test_y)
pred_y = np.concatenate(pred_y)
pred_y[pred_y >= 0.5] = 1
pred_y[pred_y < 0.5] = 0

# Accuracy, precision, recall, F1 score
acc = accuracy_score(test_y, pred_y)

print('Accuracy on test set: {:.3f}'.format(acc))
print(classification_report(test_y, pred_y))

In [0]:
weight = model.get_weights()

In [0]:
import matplotlib.pyplot as plt
plt.bar(np.abs(weight[0]).flatten())
plt.xlabel("Variable number")
plt.ylabel("Weight Coefficient")
plt.title("Weights in the Logistic Regression Model")

In [0]:
pyplot.plot(weight[0].flatten())